# Business Understanding
During this stage, you should take some time to understand the questions presented by the data from a busines perspective. Many of these are very general questions and they may not be as important or applicable as some other stages of this project for training purposes, but it's good practice to begin any project with a general understanding of what problems you are trying to solve.

Consider the following questions and answer to the best of your ability, based on the data and project documentation.

### What are the specific business objectives of this project? Define as precisely as possible.

Detecting fraudulent online transactions. Vesta detects fraud in real time and provides alerts to card holders.

### What data are available to pursue those objectives?

Transaction (target, `isFraud`, & features) and Identity (just features) data tables (csv format) are provided (train and test), and they can be joined using the TransactionID field. Additional details per Kaggle and Vesta (quoted items are from the discussion between users and Vesta while the rest was taken from a post by Vesta):

**Transaction Table**  
“It contains money transfer as well as gifted goods and service, like you booked a ticket for others, etc.”  
- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)  
“TransactionDT's first value is 86400, which corresponds to the number of seconds in a day (60 * 60 * 24 = 86400) so I think the units are seconds. Using this, we know the data spans 6 months, as the maximum value is 15811131, which would correspond to day 183.”  
- TransactionAMT: transaction payment amount in USD  
“Some of the TransactionAMT's have three decimal places to the right of the decimal point. There seems to be a link to three decimal places and a blank addr1 and addr2 field. Is it possible that these are foreign transactions and that, for example, the 75.887 in row 12 is the result of multiplying a foreign currency amount by an exchange rate?”
- ProductCD: product code, the product for each transaction  
“ProductCD isn't necessary to be a real 'product' (like one item to be added to the shopping cart). It could be any kind of service.”  
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.  
- addr: address  
“both addresses are for purchaser  
addr1 as billing region  
addr2 as billing country”  
- dist: distance  
"distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc.”  
- P_ and (R__) emaildomain: purchaser and recipient email domain  
"Certain transactions don't need s recipient, so Remaildomain is null.”  
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.  
“Can you please give more examples of counts in the variables C1-15? Would these be like counts of phone numbers, email addresses, names associated with the user? I can't think of 15.  
Your guess is good, plus like device, ipaddr, billingaddr, etc. Also these are for both purchaser and recipient, which doubles the number.”  
- D1-D15: timedelta, such as days between previous transaction, etc.  
- M1-M9: match, such as names on card and address, etc.  
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.  
“For example, how many times the payment card associated with a IP and email or address appeared in 24 hours time range, etc.  
All Vesta features were derived as numerical. Some of them are counts of orders within a clustering, a time-period or condition, so the value is finite and has ordering (or ranking). I wouldn't recommend to treat any of them as categorical. If any of them resulted in binary by chance, it maybe worth trying."  
  
Categorical Features:  
- ProductCD  
- card1 - card6  
- addr1, addr2  
- Pemaildomain Remaildomain  
- M1 - M9  

**Identity Table**  
Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions. They're collected by Vesta’s fraud protection system and digital security partners. (The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)  
  
Categorical Features:  
- DeviceType  
- DeviceInfo  
"All rows are homogeneous but columns are heterogeneous. In other words, you can think e.g. id-01 to id-20 are collected from Vesta, id-21 to id-30 are from sourceA and the rest are from sourceB.  
DeviceInfo feature is probably parsed from user agent string. That's why the value can be different things like OS version, device model or other user agent fragments."  
- id12 - id38  
“id01 to id11 are numerical features for identity, which is collected by Vesta and security partners such as device rating, ip_domain rating, proxy rating, etc. Also it recorded behavioral fingerprint like account login times/failed to login times, how long an account stayed on the page, etc. All of these are not able to be elaborated due to security partner T&C. I hope you could get the basic meaning of these features, and by mentioning them as numerical/categorical, you won't deal with them inappropriately.”

**Labeling Logic**  
"The logic of our labeling is to define a reported chargeback on the card as fraud transaction (isFraud=1) and transactions posterior to it with either user account, email address or billing address directly linked to these attributes as fraud too. If none of the above is reported and found beyond 120 days, then we define it as a legit transaction (isFraud=0).
However, in the real world, fraudulent activity might not be reported, e.g. cardholder was unaware, or forgot to report in time and beyond the claim period, etc. In such cases, supposed fraud might be labeled as legit, but we never could know of them. Thus, we think they're unusual cases and negligible portion.  
Does that mean that if a transaction, posterior to the reported fraud, has the same user account, and/or same email address, and/or billing address, will be considered as fraud = 1 too?  
It's a complicated situation - usually they will be flagged as fraud. But not all the time afterwards, you can think of one case - the billing address was found to be fraudulent in a past transaction because the credit card associated with it was stolen. But the cardholder is actually the victim, we're not going to blacklist him forever if he uses another legit card for future transaction. There're more other cases but I can't elaborate them all here.
One thing we're blacklisting for sure is the card number used for fraud."

### What resources? (programs, libraries, etc.)

Python (pandas, numpy, sklearn, xgboost, keras, matplotlib, etc.), Jupyter, Git

### What are the success criteria for each of the project's objectives? Define as precisely as possible.

Successfully predicting the probabilty that a given TransactionID is fradulent. Specifically, maximizing the area under the ROC curve (AUC) between the predicted probability of fraud and the observed target for the transactions.

### Describe the data mining problem type (regression, classification, clustering, etc.)

Problem Type:  
- Supervised  
- Classification with probabilities/scores reported as opposed to simply class predictions  
- Likely includes non-linear relationships

### What are the specific technical goals for the project?¶

- EDA to understand relationships (correlations, etc.) between the features, particularly considering the lack of context around many features  
- Feature engineering
- Feature selection
- Handling class imbalance
- Model selection
- Hyperparameter tuning
- Generate generalizable predictions with AUC as close to 1 as possible